### Dataset

In this homework, we will use the Bank Marketing dataset. Download it from [here](https://archive.ics.uci.edu/static/public/222/bank+marketing.zip).

Or you can do it with `wget`:

```bash
wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
```

We need to take `bank/bank-full.csv` file from the downloaded zip-file.  
In this dataset our desired target for classification task will be `y` variable - has the client subscribed a term deposit or not. 

### Features

For the rest of the homework, you'll need to use only these columns:

* `age`,
* `job`,
* `marital`,
* `education`,
* `balance`,
* `housing`,
* `contact`,
* `day`,
* `month`,
* `duration`,
* `campaign`,
* `pdays`,
* `previous`,
* `poutcome`,
* `y`

### Data preparation

* Select only the features from above.
* Check if the missing values are presented in the features.


In [2]:
import pandas as pd
import numpy as np

In [3]:
banks = pd.read_csv("bank-full.csv", sep=";")

In [4]:
banks.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [5]:
banks.head(2)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no


In [6]:
# select
mycols = ['age', 'job', 'marital', 'education', 'balance', 'housing',
        'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y']
dt = banks[mycols].copy()
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   balance    45211 non-null  int64 
 5   housing    45211 non-null  object
 6   contact    45211 non-null  object
 7   day        45211 non-null  int64 
 8   month      45211 non-null  object
 9   duration   45211 non-null  int64 
 10  campaign   45211 non-null  int64 
 11  pdays      45211 non-null  int64 
 12  previous   45211 non-null  int64 
 13  poutcome   45211 non-null  object
 14  y          45211 non-null  object
dtypes: int64(7), object(8)
memory usage: 5.2+ MB


In [7]:
# any missing values
dt.isnull().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

### Question 1

What is the most frequent observation (mode) for the column `education`?

- `unknown`
- `primary`
- `secondary` (23202)
- `tertiary`


In [8]:
dt.education.value_counts()

education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: count, dtype: int64

### Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset. 
In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

- `age` and `balance`
- `day` and `campaign`
- `day` and `pdays`
- `pdays` and `previous` (biggest: 0.45)


In [9]:
dt.select_dtypes(include=np.number).corr()

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.097783,-0.009120,-0.004648,0.004760,-0.023758,0.001288
balance,0.097783,1.000000,0.004503,0.021560,-0.014578,0.003435,0.016674
day,-0.009120,0.004503,1.000000,-0.030206,0.162490,-0.093044,-0.051710
duration,-0.004648,0.021560,-0.030206,1.000000,-0.084570,-0.001565,0.001203
campaign,0.004760,-0.014578,0.162490,-0.084570,1.000000,-0.088628,-0.032855
pdays,-0.023758,0.003435,-0.093044,-0.001565,-0.088628,1.000000,0.454820
previous,0.001288,0.016674,-0.051710,0.001203,-0.032855,0.454820,1.000000


### Target encoding

* Now we want to encode the `y` variable.
* Let's replace the values `yes`/`no` with `1`/`0`.


In [10]:
# binarize y in an elegant way 
# use LabelBinarizer for one-vs-all
dt.y = (dt.y == 'yes').astype(int)
dt.head(2)

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,0


In [11]:
# just check how much class imbalance is
dt.y.value_counts()

y
0    39922
1     5289
Name: count, dtype: int64

### Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
* Make sure that the target value `y` is not in your dataframe.


In [12]:
from sklearn.model_selection import train_test_split
X = dt.drop("y", axis=1)
y = dt.y
# let us hope the we get enough of positive examples
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

In [13]:
y_val.value_counts()

y
0    7972
1    1070
Name: count, dtype: int64

### Question 3

* Calculate the mutual information score between `y` and other categorical variables in the dataset. Use the training set only.
* Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the biggest mutual information score?
  
- `contact`
- `education`
- `housing`
- `poutcome` (this is largest)


### Mutual information score

The Mutual Information is a measure of the similarity between two labels of the same data.

https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.mutual_info_score.html

In [14]:
from sklearn.metrics import mutual_info_score

In [15]:
# dummy mutual info example
feature = ["technician", "entrepreneur", "self-employed", "technician", "housemaid"]
label = [0,1,1,0,0]
mutual_info_score(feature, label)

np.float64(0.6730116670092563)

In [16]:
# group columns by type
# SO: https://stackoverflow.com/questions/22470690/get-list-of-pandas-dataframe-columns-based-on-data-type
dt.columns.to_series().groupby(dt.dtypes).groups

{int64: ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'y'], object: ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']}

In [17]:
categorical =  ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

In [18]:
def calculate_mi(series):
    return mutual_info_score(series, y_train)

df_mi = X_train[categorical].apply(calculate_mi) #none are categorical
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')
df_mi.MI.round(2)

poutcome     0.03
month        0.03
contact      0.01
housing      0.01
job          0.01
education    0.00
marital      0.00
Name: MI, dtype: float64

### Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.6
- 0.7
- 0.8
- 0.9


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer #to process categorical and numerical differently

check out: https://stackoverflow.com/a/48673850 

In [20]:
def make_clf_pipe(categorical_features = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']):
    # define preprocessor
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)],
        remainder="passthrough")

    # choose classifier model
    clf = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

    # full pipeline
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', clf)])

    return pipe

In [21]:
model = make_clf_pipe()
model.fit(X_train, y_train)

/home/svetlana/.pyenv/versions/3.12.0/envs/probabl/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['job', 'marital',
                                                   'education', 'housing',
                                                   'contact', 'month',
                                                   'poutcome'])])),
                ('classifier',
                 LogisticRegression(max_iter=1000, random_state=42,
                                    solver='liblinear'))])

In [22]:
original_accuracy = model.score(X_val, y_val)
original_accuracy

0.9009068790090687

not bad!

### Question 5 

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 

Which of following feature has the smallest (absolute) difference?

- `age` - 0.0
- `balance`
- `marital`
- `previous`

> **Note**: The difference doesn't have to be positive.


In [27]:
for feature in ['age', 'balance', 'marital', 'previous']: #
    print(feature)
    # drop the feature
    print("dropping feature")
    X_train_drop = X_train.drop(feature, axis=1).copy()
    X_val_drop = X_val.drop(feature, axis=1).copy()
    # be sure to drop the feature from "categorical"
    # alternatively change the funciton above to be dependent on X df
    categorical_features = categorical.copy()
    if feature in categorical:
        categorical_features.remove(feature)
    # retrain model
    mdl = make_clf_pipe(categorical_features)
    print("fitting")
    mdl.fit(X_train_drop, y_train)
    print(mdl.named_steps.classifier.intercept_[0])
    print("evaluating")
    drop_accuracy = mdl.score(X_val_drop, y_val)
    print(f'{feature}: {original_accuracy} - {drop_accuracy} = {original_accuracy - drop_accuracy}')


age
dropping feature
fitting
-0.98840892750157
evaluating
age: 0.9009068790090687 - 0.9009068790090687 = 0.0
balance
dropping feature
fitting
-1.0110535435596826
evaluating
balance: 0.9009068790090687 - 0.9007962840079629 = 0.00011059500110588427
marital
dropping feature
fitting
-1.050090964063443
evaluating
marital: 0.9009068790090687 - 0.9010174740101747 = -0.0001105950011059953
previous
dropping feature
fitting
-0.9920076633443592
evaluating
previous: 0.9009068790090687 - 0.9011280690112807 = -0.0002211900022119906


### Question 6

* Now let's train a regularized logistic regression.
* Let's try the following values of the parameter `C`: `[0, 0.01, 0.1, 1, 10]`.
* Train models using all the features as in Q4.
* Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these `C` leads to the best accuracy on the validation set?

- 0.01
- 0.1
- 1
- 10
- 100

> **Note**: If there are multiple options, select the smallest `C`.


About C:

"C float, default=1.0
Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization."

In [28]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# check how params are called
make_clf_pipe().get_params()

In [45]:
# Grid search CV
pipe = make_clf_pipe()
parameters = {'classifier__C':[0.01, 0.1, 1, 10, 100]}
gridsearch = GridSearchCV(pipe, parameters)
gridsearch.fit(X_train_full, y_train_full) #containts train and validation set


/home/svetlana/.pyenv/versions/3.12.0/envs/probabl/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['job',
                                                                          'marital',
                                                                          'education',
                                                                          'housing',
                                                                          'contact',
                                                                          'month',
                                                                          'poutcome'])])),
                                       ('classifier',
                                        LogisticRegression(max_iter=1000,
                                                           random_state=42,
                                                           solver='liblinear'))]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100]})

In [46]:
gridsearch.cv_results_

{'mean_fit_time': array([0.62506189, 0.77377429, 0.68072481, 0.67030745, 0.59063683]),
 'std_fit_time': array([0.05067319, 0.33227919, 0.04689447, 0.07877889, 0.04137479]),
 'mean_score_time': array([0.031708  , 0.03026676, 0.03047795, 0.03058348, 0.02659454]),
 'std_score_time': array([0.00142651, 0.00061108, 0.00062159, 0.00116876, 0.00062074]),
 'param_classifier__C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0],
              mask=[False, False, False, False, False],
        fill_value=1e+20),
 'params': [{'classifier__C': 0.01},
  {'classifier__C': 0.1},
  {'classifier__C': 1},
  {'classifier__C': 10},
  {'classifier__C': 100}],
 'split0_test_score': array([0.89632292, 0.89811999, 0.89950235, 0.8985347 , 0.89922588]),
 'split1_test_score': array([0.89770528, 0.89950235, 0.90047   , 0.90212884, 0.90033177]),
 'split2_test_score': array([0.90240531, 0.90295825, 0.90295825, 0.90254354, 0.90282002]),
 'split3_test_score': array([0.9005945 , 0.90239182, 0.90266833, 0.90280658, 0.902

We got C=10 for best estimator.

In [41]:
# do it now by hand
# redo function for CV without C

def make_clf_pipe_cv(categorical_features = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']
                     , C=1.0):
    # define preprocessor
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)],
        remainder="passthrough")

    # choose classifier model
    clf = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)

    # full pipeline
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', clf)])

    return pipe

In [43]:
for C in [0.01,0.1,1,10,100]:
    pipe = make_clf_pipe_cv(C=C)
    pipe.fit(X_train, y_train)
    print(f'C={C}, accuracy {round(pipe.score(X_val, y_val),3)}')

C=0.01, accuracy 0.898
C=0.1, accuracy 0.901
C=1, accuracy 0.901
C=10, accuracy 0.901
C=100, accuracy 0.901


With rounding C=0.1 looks OK.